# K-means clustering with digits dataset

## Loading digits dataset

In [1]:
from sklearn.datasets import load_digits

digits = load_digits()
print("Digits: %i samples %i features" % digits.data.shape)

Digits: 1797 samples 64 features


## Exploratory data analysis

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')

#print(digits.target_names)

X = digits.data
y = digits.target
df = pd.DataFrame(X, columns = digits.feature_names)
#print(df.head())

#pd.plotting.scatter_matrix(df, c = y, figsize = [8, 8], s = 150, marker = 'D')